In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data/soc-sign-bitcoinotc.csv')

In [3]:
filtered_data = data.query('Evaluation >= 8')
source = list(filtered_data.Source)
target = list(filtered_data.Target)
nodes = list(set(source).union(set(target)))

In [4]:
adjacence_dict = {no:[] for no in nodes}

for index in range(len(source)):
    ori = source[index]
    dest = target[index]
    
    adjacence_dict[ori].append(dest)

In [5]:
a = [[0 for i in range(len(nodes))] for j in range(len(nodes))]

for i in range(len(nodes)):
    for j in range(len(nodes)):
        ori = nodes[i]
        dest = nodes[j]
        out_degree = len(adjacence_dict[ori])
        a[j][i] = 1/out_degree if dest in adjacence_dict[ori] else 0

a = np.matrix(a)

In [6]:
b = (1/len(nodes)) * np.matrix([[1 for i in range(len(nodes))] for j in range(len(nodes))])

In [7]:
m = 0.85 * a + 0.15 * b

In [8]:
v = (1/len(nodes)) * np.matrix([[1] for i in range(len(nodes))])

# Fórmula do pagerank
## $PR(A) = \frac{PR(B)}{L(B)} + \frac{PR(C)}{L(C)} + \frac{PR(D)}{L(D)}... \Rightarrow PR(u) = \sum_{v \in B_u} \frac{PR(B(v))}{L(v)}$

In [9]:
count = 0

def pagerank(v):
    global count
    count += 1
    
    if sum(abs(m*v-v)) > 0.001:
        return pagerank(m*v)
    return m*v

In [10]:
result = pagerank(v)
result = [cell.item(0,0) for cell in result]

In [11]:
print(count)

35


In [12]:
result_data_frame = pd.DataFrame({'id': nodes, 'PageRank': result})

In [13]:
result_data_frame.to_csv(path_or_buf='results/result.csv', index=False)
filtered_data.to_csv(path_or_buf='results/filtered-sign-bitcoinotc.csv', columns=['Source', 'Target'], index=False)